<a href="https://colab.research.google.com/github/eisbetterthanpi/vision/blob/main/resnet_ViT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## resnet_ViT

In [ ]:
# @title download
# original 10k
# # https://drive.google.com/file/d/1NkCNecLpFG3i7bo3Vl9RQSwzBpSRQ29q/view?usp=sharing
# !gdown 1NkCNecLpFG3i7bo3Vl9RQSwzBpSRQ29q -O gsv.zip
# !unzip /content/gsv.zip -d /content
# import shutil
# shutil.rmtree('/content/google_street_view/meta_data', ignore_errors=True) # delete the meta_data folder

# # clip cleaned
# # https://drive.google.com/file/d/1-xcHyVAMeTkY7SUdUyQRVAn_FFXPSDGB/view?usp=share_link
# !gdown 1-xcHyVAMeTkY7SUdUyQRVAn_FFXPSDGB -O gsv.zip
# !unzip /content/gsv.zip -d /
# !rm -R /content/gsv/.ipynb_checkpoints
# !rm -R /content/gsv/01/.ipynb_checkpoints
# !rm -R /content/gsv/02/.ipynb_checkpoints
# !rm -R /content/gsv/03/.ipynb_checkpoints
# !rm -R /content/gsv/04/.ipynb_checkpoints
# !rm -R /content/gsv/05/.ipynb_checkpoints
# !rm -R /content/gsv/06/.ipynb_checkpoints

# # gsv 70k
# # https://drive.google.com/file/d/1-7ZC29k4VxXQkpnOuLfj7Ag_SFTM4LV8/view?usp=share_link
# !gdown 1-7ZC29k4VxXQkpnOuLfj7Ag_SFTM4LV8 -O gsv.zip
# !unzip /content/gsv.zip -d /

# # # !ls
# !ls -a /content/gsv70k
# !rm -R /content/gsv70k/.ipynb_checkpoints
# # # !rm -R /content/gsv/06/.ipynb_checkpoints

# # 70k+gmap
# # https://drive.google.com/file/d/1-CZp7TbhJLeRQpbKQCyT8ofGg89Yt137/view?usp=sharing
!gdown 1-CZp7TbhJLeRQpbKQCyT8ofGg89Yt137 -O gsv.zip
!unzip /content/gsv.zip -d /
!rm -R /content/gsv70kg/.ipynb_checkpoints
!rm -R /content/gsv70kg/01/.ipynb_checkpoints
!rm -R /content/gsv70kg/02/.ipynb_checkpoints
!rm -R /content/gsv70kg/03/.ipynb_checkpoints
!rm -R /content/gsv70kg/04/.ipynb_checkpoints
!rm -R /content/gsv70kg/05/.ipynb_checkpoints
!rm -R /content/gsv70kg/06/.ipynb_checkpoints

# # https://bestasoff.medium.com/how-to-fine-tune-very-large-model-if-it-doesnt-fit-on-your-gpu-3561e50859af
!pip install bitsandbytes


In [1]:
# @title torch augment
# https://github.com/facebookresearch/vicreg/blob/main/augmentations.py
import torch
import torchvision.transforms as transforms

class TrainTransform(object):
    def __init__(self):
        # self.transform = transforms.RandomApply([transforms.Compose([
        self.transform = transforms.Compose([
                transforms.RandomPerspective(distortion_scale=0.3, p=0.5), # me
                transforms.RandomResizedCrop((400,640), scale=(0.7, 1.0), ratio=(0.8, 1.25), interpolation=transforms.InterpolationMode.BICUBIC),
                transforms.RandomHorizontalFlip(p=0.5), # 0.5
                transforms.Lambda(lambda x : torch.clamp(x, 0., 1.)), # clamp else ColorJitter will return nan https://discuss.pytorch.org/t/input-is-nan-after-transformation/125455/6
                transforms.RandomApply([transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.2, hue=0.1)], p=0.8,), # brightness=0.4, contrast=0.4, saturation=0.2, hue=0.1)], p=0.8
                transforms.RandomGrayscale(p=0.2), # 0.2
                # # transforms.RandomChoice(transforms.ColorJitter , transforms.RandomGrayscale(p=1.)
                transforms.RandomApply([transforms.GaussianBlur(kernel_size=5, sigma=(0.1, 2.0)),], p=1.0),
                # # transforms.RandomSolarize(threshold=130, p=0.5)
                # transforms.RandomErasing(p=1., scale=(0.1, 0.11), ratio=(1,1), value=(0.485, 0.456, 0.406)),
                transforms.RandomErasing(p=1., scale=(0.1, 0.11), ratio=(1,1), value=(0.5, 0.5, 0.5)),
                # transforms.ToTensor(), # ToTensored at dataset level, no need to ToTensor again
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), # normalised at dataset level. default 0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225
                ])
            # ], p=1.)

    def __call__(self, sample):
        dims = len(sample.shape)
        if dims==3: x1 = self.transform(sample) # same transforms per minibatch
        elif dims==4: x1 = transforms.Lambda(lambda x: torch.stack([self.transform(x_) for x_ in x]))(sample) # diff transforms per img in minibatch
        # x1 = self.transform(sample)
        return x1

trs=TrainTransform()


In [ ]:
# @title data (old)
# without oversampling
import torch
import torchvision
from torchvision import datasets, transforms
import numpy as np

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

dir='/content/gsv70kg'

data = datasets.ImageFolder(dir, transform=transform)
# data = datasets.ImageFolder(dir, transform=None)
torch.manual_seed(0)
train_data, test_data = torch.utils.data.random_split(data, [.9,.1])

batch_size = 16 # 64
num_batches=int(np.ceil(len(train_data)/batch_size))

# train_loader = torch.utils.data.DataLoader(train_data, sampler=train_sampler, batch_size=batch_size, num_workers=4, pin_memory=True)
# test_loader = torch.utils.data.DataLoader(test_data, sampler=test_sampler, batch_size=batch_size, num_workers=4, pin_memory=True)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, num_workers=4, pin_memory=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, num_workers=4, pin_memory=True)
del data, train_data, test_data



In [2]:
# @title data weighted
import torch
import torch.nn as nn
import torchvision
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np

# dataset has PILImage images of range [0, 1], transform them to Tensors of normalized range [-1, 1]
# transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])
# transform = transforms.Compose(transforms.ToTensor())

dir='/content/gsv70kg'

# data = datasets.ImageFolder(dir, transform=transform)
data = datasets.ImageFolder(dir, transform=None)
# torch.manual_seed(0)

# random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed_all(0)


# split data manually so that can work with weighted random sampler
# train_data, test_data = torch.utils.data.random_split(data, [.85,.15])
# https://www.scaler.com/topics/pytorch/how-to-split-a-torch-dataset/
data_size = len(data)
indices = np.arange(data_size)
np.random.shuffle(indices)
split_index = int(np.floor(0.9 * data_size))
# split_index = int(np.floor(1. * data_size))
train_idx, test_idx = indices[:split_index], indices[split_index:]
# train_idx, test_idx = indices[:split_index], indices[split_index:split_index*2]
train_data = torch.utils.data.Subset(data, train_idx)
# train_data, _ = torch.utils.data.random_split(train_data, [.1,.9])
test_data = torch.utils.data.Subset(data, test_idx)
targets = np.array(data.targets)
train_targets = targets[train_idx]
test_targets = targets[test_idx]


from torch.utils.data import Dataset
class DatasetWrap(Dataset):
    def __init__(self, dataset, transform=None):
        super(DatasetWrap, self).__init__()
        self.dataset = dataset
        self.transform = transform

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, index):
        x, y = self.dataset[index]
        if self.transform:
            x = self.transform(x)
        return x, y

# dataset wrapper in order to apply transforms to train data only
# train_data = DatasetWrap(train_data, TrainTransform()) # apply data augmentation to train dataset only
# train_data = DatasetWrap(train_data, transform) # apply transform during training to use gpu
train_data = DatasetWrap(train_data, transforms.ToTensor()) # apply transform during training to use gpu
test_data = DatasetWrap(test_data, transform)

# use batch size 16 for resnet 152/ vit with grad accumulation
# can use batch size 64 for inception v3 without grad accumulation?
batch_size = 16 # 64/16
grad_acc = 4
num_batches=int(np.ceil(len(train_data)/batch_size))

# oversampling
# https://stackoverflow.com/questions/62319228/number-of-instances-per-class-in-pytorch-dataset
from collections import Counter
class_count = torch.tensor(list(Counter(data.targets).values()))
weights=1./class_count
# weights=sum(class_count)/class_count
# print(weights)
# https://pytorch.org/docs/stable/data.html#torch.utils.data.WeightedRandomSampler

train_weight = weights[train_targets]
test_weight = weights[test_targets]
train_sampler = torch.utils.data.WeightedRandomSampler(train_weight, len(train_weight))
test_sampler = torch.utils.data.WeightedRandomSampler(test_weight, len(test_weight))
# train_loader = torch.utils.data.DataLoader(train_data, sampler=train_sampler, batch_size=batch_size, num_workers=4, pin_memory=True)
# test_loader = torch.utils.data.DataLoader(test_data, sampler=test_sampler, batch_size=batch_size, num_workers=4, pin_memory=True)
train_loader = torch.utils.data.DataLoader(train_data, sampler=train_sampler, batch_size=batch_size, pin_memory=True)
test_loader = torch.utils.data.DataLoader(test_data, sampler=test_sampler, batch_size=batch_size, pin_memory=True)
del data, train_data, test_data


# test oversampling: occurence of each class should be roughly equal
# c=0
# print(len(test_loader))
# # for batch, (x, y) in enumerate(train_loader):
# for batch, (x, y) in enumerate(test_loader):
#     print(torch.bincount(y)) # torch count number of elements with value in tensor
#     c+=1
#     if c>5: break

# import matplotlib
# matplotlib.rcParams['figure.dpi'] = 300
# def imshow(img): # display img from torch tensor
#     img = img / 2 + 0.5  # unnormalize
#     plt.axis('off')
#     npimg = img.numpy()
#     plt.imshow(np.transpose(npimg, (1, 2, 0)))
#     plt.show()

# trainiter = iter(train_loader)
# images, labels = next(trainiter)
# images=trs(images)
# imshow(torchvision.utils.make_grid(images,nrow=4))
# del trainiter

# print(labels)

# testiter = iter(test_loader)
# images, labels = next(testiter)
# imshow(torchvision.utils.make_grid(images,nrow=4))
# del testiter


In [3]:
# @title resnet
import torch
import torch.nn as nn
from torchvision import models
# # https://pytorch.org/vision/0.12/models.html#id10
model = models.resnet152(weights='DEFAULT') # 18 34 50 101 152
num_ftrs = model.fc.in_features
model.fc = nn.Sequential( # og (fc): Linear(in_features=2048, out_features=1000, bias=True)
    nn.Linear(num_ftrs, 6, bias=False),
    nn.Softmax(dim=1),
    )

# print(model)

# # model.mods = [module for k, module in model._modules.items()]
# # modules = [module for k, module in model._modules.items()]

# torch._dynamo.config.suppress_errors = True
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model = model.to(device)
model = torch.compile(model.to(device))
# model = torch.compile(model.to(device),mode='reduce-overhead')
# model = torch.compile(model.to(device),mode='max-autotune') # max needs grad ckpt

# resnet152 batch16 compile gradacc4 nogradckpt lr3e-6,3e-5


In [ ]:
# @title vit
# https://arxiv.org/pdf/2010.11929.pdf
import torch
import torch.nn as nn
from torchvision import models
# https://pytorch.org/vision/main/models/vision_transformer.html
# https://pytorch.org/vision/main/models/generated/torchvision.models.vit_b_16.html#torchvision.models.vit_b_16
# model = models.vit_l_16(weights='DEFAULT') # small vit_b_16 vit_b_32 vit_l_16 vit_l_32 vit_h_14 big
# # VisionTransformer(image_size, patch_size, num_layers, num_heads, hidden_dim, mlp_dim)
# num_ftrs = model.heads.head.in_features
# # num_ftrs = model.heads[-1].in_features
# model.heads = nn.Sequential(
#     # nn.Dropout(0.2),
#     nn.Linear(num_ftrs, 6, bias=False),
#     nn.Softmax(dim=1),
#     )


!pip install timm
# https://github.com/huggingface/pytorch-image-models/issues/908
import timm
# model = timm.create_model('vit_base_patch16_224', pretrained=True)
model = timm.create_model('vit_base_patch16_224', img_size=(400, 640), pretrained=True)
# [print(x) for x in timm.list_models('vit*',pretrained=True)]
# https://huggingface.co/google/vit-base-patch16-224
# https://github.com/huggingface/pytorch-image-models/blob/main/timm/models/vision_transformer.py
# vit_base_patch16_224 compile,no ckpt # patch_size=16, embed_dim=768, depth=12, num_heads=12
# vit_base_patch16_384
# vit_large_patch16_224 explodesgpu # patch_size=16, embed_dim=1024, depth=24, num_heads=16
# vit_large_patch16_384 # patch_size=16, embed_dim=1024, depth=24, num_heads=16

# or fine tune huge
# vit_large_patch14_224 # patch_size=16, embed_dim=1024, depth=24, num_heads=16
# vit_large_patch16_384


num_ftrs = model.head.in_features
model.head = nn.Sequential(
    nn.Linear(num_ftrs, 6, bias=False),
    nn.Softmax(dim=1),
    )
# model.set_grad_checkpointing()

# print(model.patch_embed.grid_size) # (25, 40)
# print(model.pos_embed.shape) # [1, 1001, 768]
# https://github.com/rwightman/pytorch-image-models/blob/master/timm/models/vision_transformer.py

# print(model)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model = model.to(device)
model = torch.compile(model.to(device))
# model = torch.compile(model.to(device),mode='reduce-overhead')
# model = torch.compile(model.to(device),mode='max-autotune')

# vit_base_patch16_224 batch16 maxcompile nockpt gradacc lr1e-5,1e-4


In [ ]:
# @title inception
import torch
import torch.nn as nn
from torchvision import models
# # https://pytorch.org/vision/0.12/models.html#id10
model = models.inception_v3(pretrained=True)
# https://discuss.pytorch.org/t/inception-v3-is-not-working-very-well/38296/16
# https://colab.research.google.com/github/CaoCharles/Deep-Learning-with-PyTorch/blob/master/2_Inception.ipynb
model.aux_logits = False
num_ftrs = model.fc.in_features # 2048
model.fc = nn.Sequential( # og: (fc): Linear(in_features=2048, out_features=1000, bias=True)
    nn.Linear(num_ftrs, 6, bias=False),
    nn.Softmax(dim=1),
    )
# pytorch "inception" v3 "gradient checkpointing" https://github.com/jianweif/OptimalGradCheckpointing

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model = model.to(device)
model = torch.compile(model.to(device))
# model = torch.compile(model.to(device),mode='max-autotune')

# inception batch64 compile nogradacc nogradckpt lr3e-6,3e-5


In [ ]:
# @title try

# # check model's input and output dimensions are correct
# X = torch.rand(64, 3, 32, 32, device=device)
X = torch.rand(64, 3, 400, 640, device=device)
# X = torch.rand(16, 3, 224, 224, device=device)
model.train()

# 224x224
# 16x16 / 32x32 patch
# -> 14x14=196 7x7=49 seq length
# 400x640 -> 25x40=1000 seq length

scaler = torch.cuda.amp.GradScaler()
with torch.cuda.amp.autocast():
    logits = model(X)

# modules = [module for k, module in model._modules.items()]
# for i,x in enumerate(modules):
#     print(i,x)

# logits = checkpoint_sequential(functions=modules, segments=1, input=X)

print(logits.shape)
# print(logits[0])
# print(logits[0].argmax(1))

# pred_probab = nn.Softmax(dim=1)(logits)
# y_pred = pred_probab.argmax(1)
# print(y_pred)
# del X, logits


In [ ]:
torch.cuda.empty_cache()


In [ ]:
# @title wandb
# https://docs.wandb.ai/quickstart
!pip install wandb
import wandb
wandb.login() # 487a2109e55dce4e13fc70681781de9f50f27be7
run = wandb.init(
    project="resnet50teacher",
    config={
        "model": "resnet",
        # "model": "vit",
        # "model": "inception",
        # "optim": "adamw",
        # "learning_rate": lr,
        # "epochs": epochs,
    })


In [5]:
# @title train test function
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True
scaler = torch.cuda.amp.GradScaler()
# https://github.com/prigoyal/pytorch_memonger/blob/master/models/optimized/resnet_new.py
from torch.utils.checkpoint import checkpoint, checkpoint_sequential

trs=TrainTransform() # for image augmentation during train time
# train function with automatic mixed precision
def strain(dataloader, model, loss_fn, optimizer, scheduler=None, verbose=True):
    size = len(dataloader.dataset)
    model.train()
    loss_list = []
    for batch, (x, y) in enumerate(dataloader):
        x, y = x.to(device), y.to(device)
        with torch.cuda.amp.autocast(): # automatic mixed percision
            x = trs(x) # image augmentation during train time to use gpu
            pred = model(x) # default
            # modules = [module for k, module in model._modules.items()]
            # pred = checkpoint_sequential(functions=modules, segments=1, input=x) # gradient checkpointing for resnet and inception only
            # # pred = checkpoint_sequential(functions=model.mods, segments=1, input=x)
            # print("train",pred[0])
            # print((pred.argmax(1)==y).sum().item(), pred.argmax(1).tolist(), y.tolist())
            loss = loss_fn(pred, y)
        scaler.scale(loss).backward()
        if ((batch + 1) % grad_acc == 0) or (batch + 1 == len(dataloader)): # gradient accumulation
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()
            if scheduler is not None:
                scheduler.step()
                # print("### lr: ", optimizer.param_groups[0]["lr"])

        # print(model.state_dict()['_orig_mod.bn1.running_mean'][0])
        train_loss = loss.item()/len(y)
        loss_list.append(train_loss)
        try: wandb.log({"train loss": train_loss})
        except: pass
        if batch % (size//(10* len(y))) == 0:
            # loss, current = loss.item(), batch * len(x)
            # loss, current = loss.item()/len(y), batch * len(x)
            current = batch * len(x)
            if verbose: print(f"loss: {train_loss:>7f}  [{current:>5d}/{size:>5d}]")
    return loss_list


def test(dataloader, model, loss_fn, verbose=True):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for x, y in dataloader:
            x, y = x.to(device), y.to(device)
            pred = model(x)
            # print(pred[0])
            loss = loss_fn(pred, y)
            # predicted, actual = classes[pred[0].argmax(0)], classes[y]
            test_loss += loss_fn(pred, y).item()
            # print((pred.argmax(1)==y).sum().item(), pred.argmax(1).tolist(), y.tolist())
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    # test_loss /= num_batches
    test_loss /= size
    correct /= size
    # should not use weighted rand sampler for test?
    try: wandb.log({"test loss": test_loss})
    except: pass
    if verbose: print(f"Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f}")
    return correct, test_loss


In [ ]:
# @title Lamb
# lamb optimizer (optional)
# its like adamw but with warmup-like properties built in
# https://github.com/cybertronai/pytorch-lamb/blob/master/pytorch_lamb/lamb.py
import torch
from torch.optim import Optimizer

class Lamb(Optimizer):
    """Large Batch Optimization for Deep Learning: Training BERT in 76 minutes: https://arxiv.org/abs/1904.00962
        adam (bool, optional): always use trust ratio = 1, which turns this into Adam"""
    def __init__(self, params, lr=1e-3, betas=(0.9, 0.999), eps=1e-6, weight_decay=0, adam=False): # eps=1e-8, weight_decay=0
        defaults = dict(lr=lr, betas=betas, eps=eps, weight_decay=weight_decay)
        self.adam = adam
        super(Lamb, self).__init__(params, defaults)

    def step(self, closure=None):
        loss = None
        if closure is not None: loss = closure() # closure (callable, optional): A closure that reevaluates the model and returns the loss.
        for group in self.param_groups:
            for p in group['params']:
                if p.grad is None: continue
                grad = p.grad.data
                if grad.is_sparse: raise RuntimeError('Lamb does not support sparse gradients, consider SparseAdam instad.')
                state = self.state[p] # State initialization
                if len(state) == 0:
                    state['step'] = 0
                    state['exp_avg'] = torch.zeros_like(p.data) # Exponential moving average of gradient values
                    state['exp_avg_sq'] = torch.zeros_like(p.data) # Exponential moving average of squared gradient values
                exp_avg, exp_avg_sq = state['exp_avg'], state['exp_avg_sq']
                beta1, beta2 = group['betas']
                state['step'] += 1

                # Decay the first and second moment running average coefficient
                exp_avg.mul_(beta1).add_(grad, alpha=1 - beta1) # m_t
                exp_avg_sq.mul_(beta2).addcmul_(grad, grad, value=1 - beta2) # v_t

                # Paper v3 does not use debiasing.
                # bias_correction1 = 1 - beta1 ** state['step']
                # bias_correction2 = 1 - beta2 ** state['step']
                # Apply bias to lr to avoid broadcast.
                step_size = group['lr'] # * math.sqrt(bias_correction2) / bias_correction1
                weight_norm = p.data.pow(2).sum().sqrt().clamp(0, 10)
                adam_step = exp_avg / exp_avg_sq.sqrt().add(group['eps'])
                if group['weight_decay'] != 0: adam_step.add_(p.data, alpha=group['weight_decay'])
                adam_norm = adam_step.pow(2).sum().sqrt()
                if weight_norm == 0 or adam_norm == 0: trust_ratio = 1
                else: trust_ratio = weight_norm / adam_norm
                state['weight_norm'] = weight_norm
                state['adam_norm'] = adam_norm
                state['trust_ratio'] = trust_ratio
                if self.adam: trust_ratio = 1
                p.data.add_(adam_step, alpha=-step_size * trust_ratio)
        return loss


In [ ]:
# @title LR range test
# gives insight into good LR range to use.
# for accurate results, be sure to use a new model for range test;
# also reset the model before training bec range test destroys the model!!!
# 1cycle super convergencehttps://arxiv.org/pdf/1708.07120.pdf
# # cyclic lr https://arxiv.org/pdf/1506.01186.pdf
# Note the learning rate value when the accuracy starts to
# increase and when the accuracy slows, becomes ragged, or starts to fall

# one training run of the network for a few epochs

# torch.autograd.set_detect_anomaly(True)
epochs=1
min_lr= 1e-7
max_lr= 1e-2 # 1e-2
# 152: 1e-7 - 1e-4      result 3e-7 - 3e-6
# inception: 1e-7 - 1e1      result 3e-7 - 3e-6

loss_fn = nn.CrossEntropyLoss()
# optimizer = torch.optim.AdamW(model.parameters(), lr=min_lr, betas=(0.9, 0.999), eps=1e-08, weight_decay=0.01)
optimizer = torch.optim.SGD(model.parameters(), lr=min_lr, momentum=0.9)
# optimizer = torch.optim.SGD(model.parameters(), lr=min_lr, momentum=0.)
# import bitsandbytes as bnb
# optimizer = bnb.optim.AdamW(model.parameters(), lr=min_lr, betas=(0.9, 0.999), optim_bits=8)
# optimizer = Lamb(model.parameters(), lr=min_lr, betas=(0.9, 0.999), eps=1e-08, weight_decay=3e-6)

num_batches=len(test_loader)
# num_batches=len(train_loader)

# total_steps=int(num_batches*epochs)
total_steps=int(np.ceil(num_batches/grad_acc)*epochs) # for grad accumulation

# min_lr* gamma^total_steps = max_lr
gamma = np.exp(np.log(max_lr/min_lr)/total_steps)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=gamma) # 0.75(20)-0.9(100)

gamma = np.exp(np.log(max_lr/min_lr)/(total_steps*grad_acc))
# lr_list=np.ones(total_steps)*min_lr*gamma**np.arange(total_steps)
lr_list = min_lr*gamma**np.arange(total_steps*grad_acc+1-grad_acc)

train_lst, test_lst=[],[]
# use test_loader so that the range test is shorter
# can do that since we are going to reset the model later anyways

print("lr: ", optimizer.param_groups[0]["lr"])
for i in range(epochs):
    # train_ls = train(test_loader, model, loss_fn, optimizer, scheduler)
    train_ls = strain(test_loader, model, loss_fn, optimizer, scheduler)
    # train_ls = strain(train_loader, model, loss_fn, optimizer, scheduler)
    train_lst.extend(train_ls)
print(optimizer.param_groups[0]["lr"])

# https://stackoverflow.com/a/53472966/13359815
from scipy.ndimage.filters import gaussian_filter1d
train_lstsm = gaussian_filter1d(train_lst, sigma=30)

import matplotlib.pyplot as plt
plt.plot(lr_list[:-1], train_lst[:-1])
plt.plot(lr_list[:-1], train_lstsm[:-1])
plt.xscale('log')
# plt.plot(train_lst)
# plt.plot(np.linspace(0,len(train_lst),len(test_lst)), test_lst)
plt.show()
plt.close()



In [6]:
# @title wwwwwwwww
acc_lst, train_lst, test_lst=[],[],[]
loss_fn = nn.CrossEntropyLoss()

# base_lr, max_lr = 3e-6, 3e-5 # resnet152?
base_lr, max_lr = 3e-7, 3e-6 # resnet152 batch16 gradacc4
# base_lr, max_lr = 3e-5, 3e-4 # resnet50 batch32 gradacc2
# base_lr, max_lr = 1e-5, 3e-4 # resnet18 batch128/64 gradacc1
# base_lr, max_lr = 1e-6, 1e-5 # vit
# base_lr, max_lr = 3e-6, 3e-5 # inception
# end_lr, start_lr = 1e-5, 1e-3 # 0.0001,0.1
tp=0
epochs = 8 #5 20
optimizer = torch.optim.SGD(model.parameters(), lr = base_lr, momentum=0.9)
# optimizer = torch.optim.AdamW(model.parameters(), lr=base_lr, betas=(0.9, 0.999), eps=1e-08, weight_decay=3e-6)
# import bitsandbytes as bnb # 8bit optimizer
# optimizer = bnb.optim.AdamW(model.parameters(), lr=base_lr, betas=(0.9, 0.999), optim_bits=8)
# optimizer = bnb.optim.AdamW(model.parameters(), lr=3e-6, betas=(0.9, 0.999), optim_bits=8)
# optimizer = bnb.optim.AdamW(model.parameters(), lr=3e-5, betas=(0.9, 0.999), optim_bits=8)
# optimizer = Lamb(model.parameters(), lr=base_lr, betas=(0.9, 0.999), eps=1e-08, weight_decay=3e-6)
# optimizer = Lamb(model.parameters(), lr=3e-6, betas=(0.9, 0.999), eps=1e-08, weight_decay=3e-6)
# 152 1e-5
# cnn 3e-4

div_factor = max_lr/base_lr
num_batches=len(train_loader)
# total_steps=int(num_batches*epochs)+1 # +1 to excluse uptick at the end of onecycle
total_steps=int(np.ceil(num_batches/grad_acc)*epochs +1) # /4 for when using grad accumulation
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=max_lr, total_steps=total_steps, pct_start=0.45, anneal_strategy='cos', cycle_momentum=True, base_momentum=0.85, max_momentum=0.95, div_factor=div_factor, final_div_factor=100.0, three_phase=True,)
# scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=max_lr, epochs=epochs, steps_per_epoch=num_batches, pct_start=0.45, anneal_strategy='cos', cycle_momentum=True, base_momentum=0.85, max_momentum=0.95, div_factor=div_factor, final_div_factor=10000.0, three_phase=True,)
# gamma = np.exp(np.log(end_lr/start_lr)/(num_batches*epochs)) # for scheduler step every optimizer step
# scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=gamma) # 0.75(20)-0.9(100)

# pth='/content/res15270kg.pth'
# pth='/content/drive/MyDrive/frame/res15236.pth' # B
# pth='/content/drive/MyDrive/frame/res15270kold.pth' # M
# pth='/content/drive/MyDrive/frame/res152trackf.pth' # B
# pth='/content/drive/MyDrive/frame/res152plus.pth' # B
# pth='/content/res152adamw71.pth' # ty
# pth='/content/drive/MyDrive/frame/res152Teacher_onecycle.pth' # M
# pth='/content/drive/MyDrive/frame/res18Teacher.pth' # A
# pth='/content/drive/MyDrive/frame/res50Teacher.pth' # B/
pth='/content/drive/MyDrive/frame/resnet1e1sgd.pth' # M


# pth='/content/vit.pth'
# pth='/content/drive/MyDrive/frame/vit.pth'
# pth='/content/drive/MyDrive/frame/vit3736.pth'
# pth='/content/drive/MyDrive/frame/vitadamw.pth' # M

# pth='/content/drive/MyDrive/frame/inception.pth'
# pth='/content/inception2.pth'
# pth='/content/inception.pth'


# # # to continue training
# tp, modelsd, optimsd, schedsd = torch.load(pth).values()
# # tp, modelsd, optimsd = torch.load(pth).values()
# # model.load_state_dict(modelsd)
# model.load_state_dict(modelsd,strict=False)
# optimizer.load_state_dict(optimsd)
# scheduler.load_state_dict(schedsd)



In [7]:
# @title save
from google.colab import drive
drive.mount('/content/drive')

# # pth='/content/drive/MyDrive/frame/vit.pth'
# pth='/content/drive/MyDrive/frame/res55.pth'
# pth='/content/drive/MyDrive/frame/resnet152.pth'
# pth='/content/drive/MyDrive/frame/res152373605aug.pth'
# pth='/content/res152373605aug.pth' # B
# pth='/content/drive/MyDrive/frame/res152lamb12.pth' # ty
# pth='/content/drive/MyDrive/frame/res1522.pth' # A
# pth='/content/drive/MyDrive/frame/res1522do.pth' # A
# pth='/content/drive/MyDrive/frame/res152fol6470kg.pth' # Ty B

# pth='/content/drive/MyDrive/frame/inception.pth'
# pth='/content/inception2.pth'

# pth='/content/res15270kg.pth'

# pth='/content/drive/MyDrive/frame/vit3736.pth'


# tp, modelsd, optimsd, schedsd = torch.load(pth).values()
# model.load_state_dict(modelsd)
# # optimizer.load_state_dict(optimsd)
# # scheduler.load_state_dict(schedsd)

# pth='/content/drive/MyDrive/frame/res152Teacher3e-6.pth' # M
# pth='/content/drive/MyDrive/frame/res152TeacherSGD.pth' # B

# checkpoint = { # https://discuss.pytorch.org/t/saving-model-and-optimiser-and-scheduler/52030
# 'epoch': 0,
# 'model': model.state_dict(),
# 'optimizer': optimizer.state_dict(),
# 'lr_sched': scheduler.state_dict()
# }
# torch.save(checkpoint, pth)



# pth='/content/model.pth'
# torch.save(model.state_dict(), pth)
# model.load_state_dict(torch.load(pth))
# # model.load_state_dict(torch.load("model.pth"))

# !gdown 1---4fdFbOUBTrS-VP5Va6pKowfgoU2UN -O inception2.pth
# !gdown 1visTNvWmnuV7jAm2TBiAIIrNjbOAi1Fv -O resnet152.pth
# !gdown 1-3oA1cKxgw4cfrqx079h_MKtMqRT1UFq -O res152lamb12.pth # S
# !gdown 1cxu8Qq4-FoH3W3nQdRHT5mwe0xOxZ0Kb -O res152adamw.pth # M
# !gdown 1ysJfdsvwMiWbCdkvFHwNqAUnJTtm6KbT -O res152adamw71.pth # ty
# !gdown 1VaPxGoaLjmt7K9VHi0FWbJ5efEZTLhwd -O res18teacher.pth # A

# !gdown 1NBxqdTfVjNC37R47rc79j_cl23Fd93v3 -O res50teacher.pth # B
# !gdown 1LAK8lr1_lpd7l48Z-NiB-o5JRa9k6C2V -O res50teacher.pth # M


# !gdown 1qsbDQgnPgWLIwdwsb6MKJnxAf5SRTFz5 -O inception.pth # Ty 9jun
# !gdown 1-032red_AZ4nABCXmYTvJN4s31xGctAC -O inception1615.pth # Ty

# !gdown 1SAC0TW-KNJcFqVb4kA79sQb859eT-wME -O vit.pth # A 9jun
# !gdown 1-0-GTDs5vEWnezoo-vUVagHi_niATwrN -O vit3736.pth # A 9jun





# # t, modelsd, optimsd, scheduler = torch.load('/content/drive/MyDrive/frame/resnet152.pth').values()
# t, modelsd, optimsd, scheduler = torch.load('/content/resnet152.pth').values()
# model.load_state_dict(modelsd)
# # optimizer.load_state_dict(optimsd)

# # matt152 # https://drive.google.com/file/d/1MQ0xLfHbio458uEVbn2VyMpD3bij2A4J/view?usp=sharing
# !gdown 1MQ0xLfHbio458uEVbn2VyMpD3bij2A4J -O res152.pth
# model.load_state_dict(torch.load("res152.pth"))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

import time
start = time.time()

# optimizer = bnb.optim.AdamW(model.parameters(), lr=3e-5, betas=(0.9, 0.999), optim_bits=8)
# optimizer = bnb.optim.AdamW(model.parameters(), lr=1e-7, betas=(0.9, 0.999), optim_bits=8)
# optimizer = bnb.optim.AdamW(model.parameters(), lr=0, betas=(0.9, 0.999), optim_bits=8)
# optimizer = Lamb(model.parameters(), lr=3e-5, betas=(0.9, 0.999), eps=1e-08, weight_decay=3e-6)
# optimizer = torch.optim.SGD(model.parameters(), lr=3e-6, momentum=0.9)
# optimizer.param_groups[0]["lr"]=0
# optimizer.param_groups[0]["lr"]=3e-6
optimizer.param_groups[0]["lr"]=3e-3
# optimizer.param_groups[0]["lr"]=1e-2
# optimizer.param_groups[0]["lr"]=3e-2
# optimizer.param_groups[0]["lr"]=1e-1
# resnet152 3e-3 - 1e-2     3e-2 - 1e-1?
# vit 3e-5 - 1e-4           3e-5 - 1e-4?

# scheduler = torch.optim.lr_scheduler.PolynomialLR(optimizer, total_iters=int(np.ceil(num_batches/4)*5), power=1.0)
# pth='/content/drive/MyDrive/frame/res152Tn.pth' # M
# pth='/content/drive/MyDrive/frame/res50Teacher.pth' # B/M

# scheduler = PolynomialLR(optimizer, total_iters=4, power=1.0)
# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=10**(-1/2))
# for epoch in range(5):
#     scheduler.step()
# for t in range(0,epochs):
for t in range(8):
    print(f"Epoch {t+1}\n-------------------------------")
    lr=optimizer.param_groups[0]["lr"]
    print(lr)
    # train_ls = strain(train_loader, model, loss_fn, optimizer, scheduler)
    train_ls = strain(train_loader, model, loss_fn, optimizer)
    correct, test_loss = test(test_loader, model, loss_fn)
    # train_lst.extend(train_ls)
    # test_lst.append(test_loss)
    # acc_lst.append(correct)

    checkpoint = { # https://discuss.pytorch.org/t/saving-model-and-optimiser-and-scheduler/52030
    'epoch': t+1,
    'model': model.state_dict(),
    'optimizer': optimizer.state_dict(),
    'lr_sched': scheduler.state_dict()}
    torch.save(checkpoint, pth)
    # torch.save(model.state_dict(), pth)
    end = time.time()
    print("time: ",end - start)
    start = end

# print("Done!")


# print(len(train_lst), len(test_lst))
# import matplotlib.pyplot as plt
# plt.plot(train_lst)
# plt.plot(np.linspace(0,len(train_lst),len(test_lst)), test_lst)
# plt.show()
# plt.plot(acc_lst)
# plt.show()
# plt.close()


# torch.save(model.state_dict(), "model.pth")

# resnet 18, 60/61 38.4%, 528s
# resnet 18, 58/61 39.8%, 523s
# resnet 18 compile , 58/61 40.4%, 555s
# resnet 18 compile augment , 58/61 36.4%, 1941 # augment on cpu, takes longer
# resnet 18 augment lr3e-4:3e-3, 58/61 37.7%, 1863s
# resnet 18 augment 10epoch lr1e-5:3e-4, 58/61 33.5%, 3387s
# resnet 18 compile lr1e-5:3e-4, 58/61 35.0%, 493s
# resnet 18 compile scratch lr1e-5:1e-3, 58/61 26.8%, 475s
# resnet 18 compile lr1e-5:1e-3, 55/61 47.3%, 480s
# resnet 18 compile lr1e-5:1e-3, 52/61 51.7% 503s
# resnet 18 compile lr1e-5:1e-3, unfreeze 51.0%, 550s
# resnet 18 compile lr1e-5:1e-4, unfreeze 52.7%, 518s
# resnet 34 compile lr1e-5:1e-4, unfreeze bitsadamW batch16*4
# resnet 152 compileoverhead lr3e-7:3e-6, bitsadamW batch16*4 ckpt 53.8%, 2066s
# resnet 152 from53.8% augment+cutout lr3e-7:3e-6, 53.8%, 2088s
# resnet 152 comile augment+cutout lr1e-5 /4 1epoch 48.3%, 454s
# resnet 152 comile augment+cutout lr1e-5 1epoch 48.4%, 446s
# resnet 152 comilemaxautotue augment+cutout lr1e-5 1epoch 47.7%, 448
# resnet 152 clipclean comilemax augment+cutout lr1e-5 10epoch 45.1%, 1585 *2
# resnet 152 clipclean comilemax augmax lr1e-6:3e-5 20epoch
# resnet 152 clipclean compilemax lr3e-7:1e-5,
# resnet 152 70k augment compilemax lr3e-7:3e-6 5epoch, 65.4% 11585s*5/3=19300s = 5h20m

# resnet 152 70k augment compile adamw const lr3e-6, 59.8%
# resnet 152 70k augment compile adamw const3e-6 stepdown



# resnet 152 70k augment compile lamb lr1e-2 not learning
# resnet 152 70k augment compile lamb lr3e-3 not learning
# resnet 152 70k augment compile lamb lr1e-3:1e-2 5epoch,
# 2/5epochwarmup 55.0%

# resnet 152 70kg augment1 cut1,-1 compilemax adamw lr3e-7:3e-6 1/5epoch 31.1% ty
# resnet 152 70k augment1 cut1,-1 compilemax adamw lr3e-7:3e-6 1/5epoch 28.1% A

# resnet 152 70k dataold augment1 cut1,0 compilemax adamw lr3e-7:3e-6 M nantest
# resnet 152 70kg augment1 cut1,0 compilemax adamw lr3e-7:3e-6 B
# resnet 152 70k augment1 from64.5 cut(mean) compilemax adamw lr3e-7:3e-6 4/5epochs 58.1% res152fol64 A 61.5% res152kplus M
# resnet 152 70kg augment1 from64.5 cut(mean) compilemax adamw lr3e-7:3e-6 5epochs+ 63.8% res152fol64kg res152plus B
# 73.6% 67% 66.1%
# 0 1e5 nan
# 0 3e6




# vit b16 lr1e-5 5epochs 41.3%, 466s # 4.4ram, 5.5vram
# vit l16 lr1e-5 5epochs # 32.0%, 1242s 4.5ram, 8.0vram
# vit l16 lr3e-7;1e-5 5epochs # 45.4%, 1315s 4.5ram, 8.0vram
# vit l32 lr1e-5 5epochs # .ram, .vram
# vit_large_patch16_384
# vit_base_patch16_224 maxcompile nockpt lr3e-7;1e-5 5epochs # 45.2%, 2272s 5.3ram, 11.0vram
# vit_base_patch16_224 maxcompile nockpt gradacc batch16 1e-5, 1e-2 explode
# vit_base_patch16_224 maxcompile nockpt gradacc batch16 1e-6, 1e-5 low acc, test nan
# vit_base_patch16_224 maxcompile nockpt gradacc batch16 3e-7, 3e-6 4/5 epoch50%
# vit_base_patch16_224 maxcompile nockpt gradacc batch16 1e-6, 1e-5 0.5aug
# vit_base_patch16_224 maxcompile nockpt gradacc batch16 aug1 noblur Lamb lr1e-5 vitadamw B
# 1h per epoch
# 0 1e5 nan
# 0 1e6




# inception
# inception og 10kclean 1e-7, 3e-5 batch64 nockpt 5epochs 0.020348  51.4%, 0.024312 20m; 1st 40.5% max 52.1
# inception hid 10kclean 1e-7, 3e-5 batch64 nockpt 1/5epochs 0.019635 53.6%, 0.024120 1st 32.8%, max 53.6%
# inception og 10kclean 1e-7, 1e-4 batch64 nockpt 2/10epochs 0.020348  0.020923 39.9%, 0.026105
# inception og 70kg 1e-7 nope
# inception og 70kg 3e-6 3e-5 54.8%
# inception og 70kg 1e-6, 1e-5
# inception og 70kg 3e-6 3e-5 0.5aug
# adamw 3e-5 A 51.8% 53.3%
# inception 70kg 3e-5 S
# inception 70kg 3e-5 3e 61.0% 30m per epoch



Epoch 1
-------------------------------
0.1
loss: 0.112065  [    0/61144]
loss: 0.113565  [ 6112/61144]
loss: 0.109091  [12224/61144]


In [ ]:

def deactivate_batchnorm(m):
    if isinstance(m, nn.BatchNorm2d):
        m.reset_parameters()
        m.eval()
        with torch.no_grad():
            m.weight.fill_(1.0)
            m.bias.zero_()

model.apply(deactivate_batchnorm)


In [ ]:
# correct, test_loss = test(test_loader, model, loss_fn)
correct, test_loss = test(train_loader, model, loss_fn) # dont forget to normalise train data
print(correct, test_loss)
# res152 70 Accuracy: 69.5%, Avg loss: 0.084133 ; trainloader:Accuracy: 68.6%, Avg loss: 0.084691

# # https://discuss.pytorch.org/t/model-predictions-changing-with-no-grad-and-eval/126543/2
# self.drop_layer = nn.Dropout(p = .5)
# out = F.dropout(x, training=self.training)
# # https://discuss.pytorch.org/t/trained-resnet-doesnt-work-in-eval-mode-behaves-strangely/121242/8
# self.bn = torch.nn.BatchNorm2d(input_features,track_running_stats=False)

# res50 Accuracy: 36.3%, Avg loss: 0.052037 Accuracy: 45.6%, Avg loss: 0.049405
# res18 Accuracy: 62.3%, Avg loss: 0.044281



Accuracy: 45.6%, Avg loss: 0.049405
0.4562689511024758 0.049405341554549245


In [ ]:
# @title confusion matrix
# for plotting confusoin matrix
# left vertical axis is actual, bottom horizontal axis is predicted
from sklearn.metrics import confusion_matrix
from matplotlib import pyplot as plt
import seaborn as sns

def confusion(dataloader, model, loss_fn, verbose=True):
    model.eval()
    y_true, y_pred = torch.empty(0, device=device), torch.empty(0, device=device)
    for x, y in dataloader:
        x, y = x.to(device), y.to(device)
        with torch.no_grad():
            pred = model(x)
        # print((pred.argmax(1)==y).sum().item(), pred.argmax(1).tolist(), y.tolist())
        y_pred = torch.cat((y_pred, pred.argmax(1)), 0)
        y_true = torch.cat((y_true, y), 0)
    return y_true, y_pred

y_true, y_pred = confusion(test_loader, model, loss_fn)

y_true, y_pred = y_true.cpu(), y_pred.cpu()
cm = confusion_matrix(y_true=y_true, y_pred=y_pred)
cmn = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
fig, ax = plt.subplots(figsize=(5,5))
sns.heatmap(cmn, annot=True, fmt='.2f', xticklabels=['1','2','3','4','5','6',], yticklabels=['1','2','3','4','5','6',])
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()



In [ ]:
# @title collect unconfident preds
import os
from PIL import Image

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

t=0
for cls in range(1,7):
    img_dir = '/content/gsv70kg/0'+str(cls)
    for filename in os.listdir(img_dir):
        img_file=os.path.join(img_dir, filename)
        image = Image.open(img_file).convert("RGB")
        with torch.no_grad():
            with torch.cuda.amp.autocast(): # automatic mixed percision
                x = transform(image).to(device).unsqueeze(0)
                y = torch.tensor(cls-1, device=device)
                pred = model(x)

        p = pred.max(1)
        # s.append((p, img_file, y.item(), pred.argmax(1)))
        s.append((p, img_file))

        t+=1
        # if t >=5: break


s.sort() # lowest confidence to highest confidence

sa=[x[0] for x in s] # p
sb=[x[1] for x in s] # img_file

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
matplotlib.rcParams['figure.dpi'] = 300

def imshow(img):
    # img = img / 2 + 0.5  # unnormalize
    npimg = img.numpy()
    plt.axis('off')
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

from torchvision import datasets, transforms
c=3*64
images = [Image.open(img_file).convert("RGB") for img_file in sb[c:c+64]]
images = [transforms.ToTensor()(im) for im in images]
imshow(torchvision.utils.make_grid(images,nrow=8))


# print(type(sa[0]))
# print(sa[0].values.item())

# for i,x in enumerate(sa[:256]):
#     # print(x)
#     print(i, x.values.item(), sb[i])




38767 67938 57.062321528452415


In [ ]:
# @title collect misclassified
import os
from PIL import Image


transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

rongs=[[[] for i in range(6)] for j in range(6)]


t=0
c=0
for cls in range(1,7):
    img_dir = '/content/gsv70kg/0'+str(cls)
    for filename in os.listdir(img_dir):
        # name = os.path.splitext(filename)[0]
        img_file=os.path.join(img_dir, filename)
        image = Image.open(img_file).convert("RGB")
        with torch.no_grad():
            with torch.cuda.amp.autocast(): # automatic mixed percision
                x = transform(image).to(device).unsqueeze(0)
                y = torch.tensor(cls-1, device=device)
                pred = model(x)

        p = pred.argmax(1)
        # print("p, y", p, y)
        if p != y:
            # print(filename)
            c+=1
            rongs[y][p].append(filename)

        t+=1
        # if t >=5: break

print(c,t,c*100/t) # correct, total, accuracy
# 38767 67938 57.062321528452415

# print(rongs)
[print([len(s) for s in l]) for l in rongs]


# display misclassified images

# def imshow(img):
#     img = img / 2 + 0.5  # unnormalize
#     npimg = img.numpy()
    # plt.imshow(np.transpose(npimg, (1, 2, 0)))
    # plt.show()


# combi = torch.combinations(torch.arange(0,6), with_replacement=True)
combi = [[i,j] for i in range(6) for j in range(6)]
# print(combi)

for i, j in combi:
    print("actual: ", i, "predicted: ", j)
    c=0
    images=torch.empty(0)
    plt.figure(figsize=(16, 16))
    plt.axis('off')
    for filename in rongs[i][j]:
    # for filename in rongs[5][0]:
        # print(filename)
        cls=filename[1]
        img_dir = '/content/gsv70kg/0'+str(cls)
        img_file=os.path.join(img_dir, filename)
        image = Image.open(img_file)#.convert("RGB")
        image = transform(image).unsqueeze(0)
        images=torch.cat((images,image),0)

        c+=1
        if c>15: break
    try: imshow(torchvision.utils.make_grid(images,nrow=4))
    except: continue
# except: pass



In [ ]:
# pytorch release gpu ram after training
# https://discuss.pytorch.org/t/free-all-gpu-memory-used-in-between-runs/168202/2
torch.cuda.empty_cache()


In [ ]:
# @title plot lr scheduler
lr_lst=[]
total_steps=100
base_lr, max_lr = 3e-5, 3e-4 # resnet50 batch32 gradacc2

optimizer = bnb.optim.AdamW(model.parameters(), lr=base_lr, betas=(0.9, 0.999), optim_bits=8)

scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=max_lr, total_steps=total_steps, pct_start=0.45, anneal_strategy='cos', cycle_momentum=True, base_momentum=0.85, max_momentum=0.95, div_factor=div_factor, final_div_factor=100.0, three_phase=True,)

for t in range(total_steps):
    lr=optimizer.param_groups[0]["lr"]
    lr_lst.append(lr)
    scheduler.step()
plt.plot(lr_lst)
